# 📊 구조화된 출력(Structured Output) 생성

참조 링크 : https://learn.microsoft.com/en-us/agent-framework/agents/structured-output?pivots=programming-language-python

## 🎯 개요

에이전트를 사용하여 **구조화된 출력**을 생성하는 방법을 알아봅니다.

---
**중요 : 지원 에이전트 타입**

모든 [에이전트 유형](SupportedAgentType.ipynb)이 구조화된 출력을 지원하는 것은 **아닙니다**.   
`Agent`는 호환 가능한 채팅 클라이언트와 함께 사용할 때 구조화된 출력을 지원합니다.

## 📝 Step 1: 사용자 모델 정의

에이전트로부터 얻고자 하는 **출력의 구조**를 나타내는 Pydantic 모델을 정의합니다.

In [ ]:
from pydantic import BaseModel

class PersonInfo(BaseModel):
    """사용자에 대한 정보입니다. 👤ℹ️"""
    name: str | None = None
    age: int | None = None
    occupation: str | None = None

## 🤖 Step 2: 에이전트 생성

Azure OpenAI Chat 클라이언트를 사용하여 에이전트를 생성합니다.

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# Create the agent using Azure OpenAI Chat Client
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
   name="HelpfulAssistant",
   instructions="당신은 텍스트에서 사람 정보를 추출하는 도움이 되는 어시스턴트입니다. 📝✨"#
)


## 🎁 Step 3: 구조화된 데이터 접근

Response의 **`value`** 속성을 통해 PersonInfo 모델 인스턴스에 직접 액세스할 수 있습니다.

In [ ]:
query = "35세, 소프트웨어 개발자인 김흥부에 대한 정보를 제공해 주세요"
print(f"프롬프트 : {query}")

response = await agent.run(query, options={"response_format": PersonInfo})

if structured_data := response.value:
    print("구조화된 출력:")
    print(f"이름: {structured_data.name}")
    print(f"나이: {structured_data.age}")
    print(f"직업: {structured_data.occupation}")
else:
    print(f"응답을 파싱하지 못했습니다: {response.text}")

## 🌊 Step 4: 스트리밍에서 구조화된 출력 처리

스트리밍 시 에이전트 응답은 **일련의 업데이트**로 스트리밍됩니다. 구조화된 출력을 얻으려면 스트리밍을 완료한 다음, get_final_response()를 호출하여 최종 파싱된 값을 얻을 수 있습니다.

In [ ]:
from agent_framework import AgentResponse

query = "그래픽 디자이너이고 서울에 사는 28살 이제니에 대해서 알려주세요"
print(f"프롬프트 : {query}")

# ResponseStream을 사용하여 실시간으로 업데이트 스트리밍하기
stream = agent.run(query, stream=True, options={"response_format": PersonInfo})
async for update in stream:
    if update.text:
        print(f"업데이트: {update.text}", end="\n", flush=True)
print("-"*60)

# get_final_response() 메서드는 구조화된 출력이 파싱된 AgentResponse를 반환합니다.
result = await stream.get_final_response()

if structured_data := result.value:
    print("구조화된 출력 (ResponseStream을 사용하는 스트리밍):")
    print(f"이름: {structured_data.name}")
    print(f"나이: {structured_data.age}")
    print(f"직업: {structured_data.occupation}")
else:
    print(f"응답을 파싱하지 못했습니다: {result.text}")

다음은 일반 출력과 스트리밍을 사용하는샘플 예시 코드입니다.

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

from agent_framework.openai import OpenAIResponsesClient
from pydantic import BaseModel

"""
OpenAI Responses Client with Structured Output Example

This sample demonstrates using structured output capabilities with OpenAI Responses Client,
showing Pydantic model integration for type-safe response parsing and data extraction.
"""


class OutputStruct(BaseModel):
    """A structured output for testing purposes."""

    city: str
    description: str


async def non_streaming_example() -> None:
    print("=== Non-streaming example ===")

    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
        name="CityAgent",
        instructions="You are a helpful agent that describes cities in a structured format.",
    )

    query = "Tell me about Paris, France"
    print(f"User: {query}")

    result = await agent.run(query, options={"response_format": OutputStruct})

    if structured_data := result.value:
        print("Structured Output Agent:")
        print(f"City: {structured_data.city}")
        print(f"Description: {structured_data.description}")
    else:
        print(f"Failed to parse response: {result.text}")


async def streaming_example() -> None:
    print("=== Streaming example ===")

    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
        name="CityAgent",
        instructions="You are a helpful agent that describes cities in a structured format.",
    )

    query = "Tell me about Tokyo, Japan"
    print(f"User: {query}")

    # Stream updates in real time using ResponseStream
    stream = agent.run(query, stream=True, options={"response_format": OutputStruct})
    async for update in stream:
        if update.text:
            print(update.text, end="", flush=True)
    print()

    # get_final_response() returns the AgentResponse with structured output parsed
    result = await stream.get_final_response()

    if structured_data := result.value:
        print("Structured Output (from streaming with ResponseStream):")
        print(f"City: {structured_data.city}")
        print(f"Description: {structured_data.description}")
    else:
        print(f"Failed to parse response: {result.text}")


async def main() -> None:
    print("=== OpenAI Responses Agent with Structured Output ===")

    await non_streaming_example()
    await streaming_example()


if __name__ == "__main__":
    await main()